# Deep Reinforcement Learning

In [1]:
# Arian Noori 401106663
# Barbod ShahrAbadi 401106125

 Reinforcement Learning (RL) is an approach wherein an agent learns to make sequential decisions by interacting with an environment. The objective is for the agent to maximize the cumulative reward it receives over time.
 The agent goes through this process by repeatedly evaluating the consequences of its actions, trying to select actions that lead to better outcomes.

To do this, we will use Gym, an platform for developing and comparing reinforcement learning algorithms. Gym provides an interface for interacting with different environments, it accepts actions from agents and plays them out in an environment, providing rewards.


## Environment

We will be using `CartPole` environment from gym's library for this assignment.  In this environment, a pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The pendulum is placed upright on the cart and the goal is to balance the pole by applying forces in the left and right direction on the cart.

You can use the code below to run an instance of a random agent in this environment and see the results.

In [28]:
from IPython.display import HTML
from base64 import b64encode

def show_video(path):
    mp4 = open(path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
          <source src="%s" type="video/mp4">
    </video>
    """ % data_url)


In [4]:
!pip install gym[atari,accept-rom-license] -qq
!pip install imageio -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 27.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [5]:
import cv2
import gym
import imageio
import numpy as np
from gym import spaces

We use `gym.make()` to make an instance of a certain environemtn. We can then use `.step()` method which accepts an action as input and performs it. Before that we reset the environment to its initial state by using `.reset()` method.

In [29]:
env_name = 'CartPole-v1'

# Create an instance of the CartPole environment

env = gym.make(env_name)
env.reset()

frames = []

for _ in range(500):
    action = env.action_space.sample()

    obs, reward, done, _ = env.step(action)

    # render this frame and add the list of frames
    frames.append(env.render(mode='rgb_array'))

    if done:
        env.reset()

env.close()
imageio.mimsave('./cartpole.mp4', frames, fps=25)


In [30]:
show_video('./cartpole.mp4')

As you can see, the cart fails to keep the balance of the pole. In the next section we will train an agent to learn how to perform this task.

## Algorithm
We will be using A2C algorithm.

Advantage Actor-Critic (A2C) is a reinforcement learning algorithm.
It consists of an actor (which predicts the best action based on the current state) and a critic (which estimates the state's value function to measure expected future rewards).

We will implement this together step by step.




In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions.categorical import Categorical
import numpy as np
import gym
from collections import deque
from tqdm import tqdm
import imageio
from IPython.display import HTML
from base64 import b64encode


## Neural Network

Here we design a simple feed forward model to embed the observation from the environment to a hidden layer. We then use two fully connected layers on top of the hidden layer, to predict the next action and estimate the value of current state. This acts as both actor, and critic.

You need to implement the convolutional neural network.


In [9]:
class ActorCritic(nn.Module):
    def __init__(self, input_size, hidden_size, num_outputs):
        super(ActorCritic, self).__init__()
        self.fully_connected = nn.Linear(input_size, hidden_size)
        self.actor = nn.Linear(hidden_size, num_outputs)
        self.critic = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = F.relu(self.fully_connected(x))
        action_probs = F.softmax(self.actor(x), dim=-1)
        value = self.critic(x)
        return action_probs, value


## A2C

The A2C algorithm aims to jointly train both the actor and the critic to improve the policy. It does this by updating the parameters
of the actor to increase the likelihood of good actions and updating the parameters
of the critic to better estimate the value function.

In each iteration A2C plays the until it ends. During this time it records log probabality of actions, rewards, and predicted values in each step. These values will be used to update the model at the end of this trajectory.

The actor is updated using the objective below:

$$ L_{\text{actor}} = -\log \pi(a|s;\theta) \times A(s, a) $$
Where advantage is calculated as:
$$A(s, a) = Q(s, a) - V(s) $$

Namely the function $Q(s,a)$ is the estimated value of taking action
$a$
 in state
$s$.
$V(s)$ is the predicted value of our critic.

This loss function aims to improve the probability of playing actions that result in higher rewards.

As for the critic the loss function is defined as a simple mean square loss between actual value of an state and the predicted one:

$$ L_{\text{critic}} = \frac{1}{2} ( R - V(s))^2 $$

In [10]:
class A2CAgent:
    def __init__(self, env, num_episodes=2000, max_steps=500, gamma=0.99, lr=5e-4, hidden_size=256):
        self.env = env
        self.num_episodes = num_episodes
        self.max_steps = max_steps
        self.gamma = gamma
        self.lr = lr
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.policy_net = self._build_network(hidden_size)
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=lr)
        self.critic_loss = nn.MSELoss()

    def _build_network(self, hidden_size):
        input_size = self.env.observation_space.shape[0]
        num_outputs = self.env.action_space.n
        return ActorCritic(input_size, hidden_size, num_outputs).to(self.device)

    def choose_action(self, state):
        state = self._to_tensor(state)
        action_probs, _ = self.policy_net(state)
        action_dist = Categorical(action_probs)
        action = action_dist.sample()
        return action.item(), action_dist.log_prob(action)

    def _to_tensor(self, state):
        if not isinstance(state, torch.Tensor):
            state = torch.FloatTensor(state).to(self.device)
        return state

    def compute_returns(self, rewards):
        R = 0
        returns = []
        for r in reversed(rewards):
            R = r + self.gamma * R
            returns.insert(0, R)
        return returns

    def train(self):
        episode_rewards = []
        with tqdm(range(self.num_episodes)) as pbar:
            for episode in pbar:
                episode_reward, log_probs, values, rewards = self._run_episode()
                episode_rewards.append(episode_reward)

                returns = self.compute_returns(rewards)
                returns = torch.FloatTensor(returns).to(self.device)

                loss = self._compute_loss(log_probs, values, returns)
                self._optimize_model(loss)

                pbar.set_description(f"Episode {episode}, Reward: {episode_reward:.2f}")

        self.env.close()
        return episode_rewards

    def _run_episode(self):
        state = self.env.reset()
        episode_reward = 0
        log_probs = []
        values = []
        rewards = []

        for step in range(self.max_steps):
            state = self._to_tensor(state)
            action, log_prob = self.choose_action(state)
            value = self.policy_net(state)[1]

            next_state, reward, done, _ = self.env.step(action)
            log_probs.append(log_prob)
            values.append(value)
            rewards.append(reward)
            episode_reward += reward

            if done:
                break

            state = next_state

        return episode_reward, log_probs, values, rewards

    def _compute_loss(self, log_probs, values, returns):
        log_probs = torch.stack(log_probs)
        values = torch.stack(values).squeeze()
        advantage = returns - values

        actor_loss = - (log_probs * advantage.detach()).mean()
        critic_loss = self.critic_loss(values, returns)
        return actor_loss + critic_loss

    def _optimize_model(self, loss):
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

Define the model and set hyperparameters.

In [14]:
env_name = 'CartPole-v1'
env = gym.make(env_name)

num_episodes = 1000
max_steps = 500
lr = 1e-3
hidden_size = 256

a2c_model = A2CAgent(env, num_episodes, max_steps, lr=lr, hidden_size=hidden_size)


Train the model.

In [15]:
rewards = a2c_model.train()

Episode 999, Reward: 500.00: 100%|██████████| 1000/1000 [07:51<00:00,  2.12it/s]


## Evaluation

Use the `choose_action` method of the trained agent to evaluate its performance.

In [31]:

'''env = gym.make(env_name, render_mode='rgb_array')

model = a2c_model

num_episodes = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# complete the loop and gather the episode reward of each episode as well the frames of the first episode
frames = []
episode_rewards = []

for i in range(num_episodes):
    state = env.reset()
    episode_reward = 0

    state = torch.FloatTensor(state).to(device)


    for step in range(max_steps):
      action , _ = model.choose_action(state)
      next_state , reward , done , _ = env.step(action)
      if(i == 1):
        frames.append(env.render(mode='rgb_array'))

      state = torch.FloatTensor(next_state).to(device)
      episode_reward += reward


    episode_rewards.append(episode_reward)
    print(f"Episode Reward: {episode_reward}")
env.close()

episode_rewards = np.array(episode_rewards)
print(f"Average Reward over {num_episodes} episodes: {np.mean(episode_rewards)}")
print(frames)
imageio.mimsave('./test.mp4' , frames , fps=25)'''

env = gym.make(env_name)

model = a2c_model

num_episodes = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# complete the loop and gather the episode reward of each episode as well the frames of the first episode
frames = []
episode_rewards = []

for i in range(num_episodes):
    state = env.reset()
    episode_reward = 0

    state = torch.FloatTensor(state).to(device)


    for step in range(max_steps):
      action , _ = model.choose_action(state)
      next_state , reward , done , _ = env.step(action)
      if(i == 1):
        frames.append(env.render(mode='rgb_array'))

      state = torch.FloatTensor(next_state).to(device)
      episode_reward += reward


    episode_rewards.append(episode_reward)
    print(f"Episode Reward: {episode_reward}")
env.close()

episode_rewards = np.array(episode_rewards)
print(f"Average Reward over {num_episodes} episodes: {np.mean(episode_rewards)}")
imageio.mimsave('./test.mp4', frames, fps=25)

Episode Reward: 500.0
Episode Reward: 500.0
Episode Reward: 430.0
Episode Reward: 500.0
Episode Reward: 500.0
Episode Reward: 500.0
Episode Reward: 500.0
Episode Reward: 489.0
Episode Reward: 500.0


Episode Reward: 500.0
Average Reward over 10 episodes: 491.9


In [32]:
show_video('./test.mp4')